In [80]:
# read the file 'dataset_reg_pair_filled.csv' and generate synthetic data
# first read the file, then make a list of the source training images
# then for each image, generate 10 synthetic images with random affine transformation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import math
import csv
from PIL import Image
import shutil

import torch
import torchvision.transforms as transforms

# read the file
df = pd.read_csv('Dataset/dataset_reg_pair_filled.csv')
print(df.shape)
df.head()

(5040, 9)


,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory
0,2011248_20161215__L_b2.jpg,NaN,2011248_20161215__L_b1.jpg,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb
1,2011248_20161215__L_b2.jpg,NaN,2011248_20161215__L_b3.jpg,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb
2,2011248_20161215__L_b1.jpg,NaN,2011248_20161215__L_b3.jpg,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb
3,2011248_20161215__L_c2.jpg,NaN,2011248_20161215__L_c1.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc
4,2011248_20161215__L_c2.jpg,NaN,2011248_20161215__L_c3.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc


In [81]:
# make a list of the unique source training images that has 'training' = 0
# each image path consists of image directory, image name

df_train = df[df['training'] == 0]

# create a new df consists of image directory and image name concatenated
df_train['image_path'] = df_train['Directory'] + '/' + df_train['source']
df_train.head()


/tmp/ipykernel_21759/1623562252.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['image_path'] = df_train['Directory'] + '/' + df_train['source']


,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory,image_path
3,2011248_20161215__L_c2.jpg,NaN,2011248_20161215__L_c1.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc,Dataset/Dataset-processed/15-12-2559/2011248/L...
4,2011248_20161215__L_c2.jpg,NaN,2011248_20161215__L_c3.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc,Dataset/Dataset-processed/15-12-2559/2011248/L...
5,2011248_20161215__L_c1.jpg,NaN,2011248_20161215__L_c3.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc,Dataset/Dataset-processed/15-12-2559/2011248/L...
6,2011248_20161215__R_b2.jpg,NaN,2011248_20161215__R_b3.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Rb,Dataset/Dataset-processed/15-12-2559/2011248/R...
7,2011248_20161215__R_b2.jpg,NaN,2011248_20161215__R_b1.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Rb,Dataset/Dataset-processed/15-12-2559/2011248/R...


In [82]:
print(df_train.shape)
# make a list of the unique values in the column 'image_path'

image_list = df_train['image_path'].unique()
print(len(image_list))
print(image_list[0])


(4132, 10)
920
Dataset/Dataset-processed/15-12-2559/2011248/Lc/2011248_20161215__L_c2.jpg


In [83]:
min_translation = 0.1  # Minimum translation in pixels
max_translation = 0.2  # Maximum translation in pixels
max_rotation = 10  # Maximum rotation in degrees
max_shear = 10  # Maximum shear in degrees
min_scale = 0.9  # Minimum scale factor
max_scale = 1.1  # Maximum scale factor

# This file need to be edited first

In [84]:
def generate_affine_transformed_images_multiple(img_list, csv_file, output_dir, num_images=5, modify=False):
    # delete all files and subdirectories in the output directory
    shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # Initialize the CSV file with a header
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["source", "target", "M[0]", "M[1][0]", "M[1][1]", "M[2]", "M[3][0]", "M[3][1]", "SourceImage"])

    # Define the affine transformation
    affine_transform = transforms.RandomAffine(degrees=(-max_rotation, max_rotation), translate=(min_translation, max_translation),
                                               shear=(-1*max_shear, max_shear), scale=(min_scale, max_scale))

    # Loop over the images, read the image, 
    # apply affine transformation and save it
    for i, img_path in enumerate(img_list):
        # Read the image as grayscale using cv2
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        for j in range(num_images):
            # Apply affine transformation using PyTorch
            img_transformed = affine_transform(Image.fromarray(image))

            # Convert the transformed image to a numpy array
            img_transformed = np.array(img_transformed)

            # Save original image
            original_image_path = os.path.join(output_dir, f"img_{i}_original.png")
            cv2.imwrite(original_image_path, image)

            # add some noise to the transformed image and save it
            img_transformed = img_transformed + np.random.normal(0, 0.2, img_transformed.shape)
            if modify: # if modify is True, then add some noise and intensity change to the transformed image
                img_transformed = img_transformed + np.random.normal(0, 1, img_transformed.shape)
                img_transformed = img_transformed * random.uniform(0.5, 1.5)

            transformed_image_path = os.path.join(output_dir, f"img_{i}_transformed_{j}.png")
            cv2.imwrite(transformed_image_path, img_transformed)

            # Save the affine parameters to a CSV file
            # ref: https://programtalk.com/python-more-examples/torchvision.transforms.RandomAffine.get_params/
            M = affine_transform.get_params(degrees=(-max_rotation, max_rotation), translate=(min_translation, max_translation),
                                            scale_ranges=(min_scale, max_scale), shears=(-1*max_shear, max_shear), img_size=image.shape)
            # print(M)
            print(M[0], M[1][0], M[1][1], M[2], M[3][0], M[3][1])
            
            with open(csv_file, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([original_image_path, transformed_image_path, 
                                M[0], M[1][0], M[1][1], M[2], M[3][0], M[3][1], img_path])

    print(f"Generated {(i + 1)*num_images} images")



In [85]:
# Define parameters
image_size = (512, 512)  # Size of the images
output_dir = "Dataset/synthetic_eye_dataset_train"  # Output directory
os.makedirs(output_dir, exist_ok=True)

# plot_dir = "Dataset/synthetic_eye_dataset_train/plot"
# os.makedirs(plot_dir, exist_ok=True)

# generate synthetic images for each source training image
generate_affine_transformed_images_multiple(image_list,'Dataset/dataset_eye_synth_train.csv', output_dir, num_images=1, modify=True)

2.421005964279175 -17 17 0.943600058555603 -4.766029357910156 0.0
-3.3937859535217285 38 -65 0.9512834548950195 -3.6575400829315186 0.0
-9.55433464050293 45 33 0.9259515404701233 -2.9877960681915283 0.0
-5.201008319854736 -26 -59 0.9990115165710449 1.674560308456421 0.0
-4.745433330535889 -16 -42 1.0792349576950073 2.110147476196289 0.0


-0.8503806591033936 -2 -101 0.9978353381156921 -3.6406826972961426 0.0
-4.820652961730957 -42 82 1.016185998916626 -4.125497341156006 0.0
-6.596777439117432 42 65 0.942205011844635 -2.030271291732788 0.0
7.155610084533691 0 -94 1.063134789466858 7.02269172668457 0.0
6.702721118927002 -46 64 0.9705656170845032 1.6780352592468262 0.0
6.541099548339844 2 -37 0.9434751272201538 -1.2102746963500977 0.0
8.78456974029541 43 28 0.9089348912239075 6.860686302185059 0.0
-9.565535545349121 35 55 0.9494622349739075 -6.372675895690918 0.0
-8.279995918273926 -29 -67 0.9674242734909058 2.772001028060913 0.0
-1.6613459587097168 10 -5 1.0081628561019897 -9.118815422058105 0.0
1.894303560256958 -30 24 1.0127977132797241 -6.947688102722168 0.0
4.726058006286621 51 -77 1.0006182193756104 -8.960477828979492 0.0
0.061697959899902344 -33 -15 0.9409885406494141 7.599698066711426 0.0
-1.0597968101501465 -16 17 0.932142972946167 -2.6068294048309326 0.0
3.8798141479492188 -27 -17 1.093489170074463 -7.60325908660

In [86]:
# # Define parameters
# image_size = (512, 512)  # Size of the images
# output_dir = "Dataset/synthetic_eye_dataset_train_multiple"  # Output directory
# os.makedirs(output_dir, exist_ok=True)

# # plot_dir = "Dataset/synthetic_eye_dataset_train_multiple/plot"
# # os.makedirs(plot_dir, exist_ok=True)


# # generate synthetic images for each source training image
# generate_affine_transformed_images_multiple(image_list,'dataset_eye_synth_train_multiple.csv', output_dir, num_images=2, modify=True)

In [87]:
# do the same for the test images
df_test = df[df['training'] == 1]
df_test['image_path'] = df_test['Directory'] + '/' + df_test['source']
image_list_test = df_test['image_path'].unique()

# Define parameters
image_size = (512, 512)  # Size of the images
output_dir = "Dataset/synthetic_eye_dataset_test"  # Output directory
os.makedirs(output_dir, exist_ok=True)

# plot_dir = "Dataset/synthetic_eye_dataset_test/plot"
# os.makedirs(plot_dir, exist_ok=True)

# generate synthetic images for each source test image
generate_affine_transformed_images_multiple(image_list_test, 'Dataset/dataset_eye_synth_test.csv', output_dir, num_images=1)



-0.07475137710571289 14 -42 0.9790862798690796 0.008431673049926758 0.0
4.631495475769043 35 79 0.9069253206253052 6.544379234313965 0.0
8.712830543518066 5 72 1.0813900232315063 5.676751136779785 0.0
-5.001002311706543 -26 44 0.9222790002822876 6.466979026794434 0.0


/tmp/ipykernel_21759/1326887381.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['image_path'] = df_test['Directory'] + '/' + df_test['source']


-6.730234622955322 -11 3 1.0197808742523193 -4.423927307128906 0.0
-2.4757421016693115 39 -80 1.0068297386169434 -4.172519683837891 0.0
-5.076148509979248 -20 52 0.9536213278770447 -8.208719253540039 0.0
7.85147762298584 -29 31 1.099305510520935 -3.9848744869232178 0.0
-9.23139476776123 -50 23 0.9541430473327637 -4.987735748291016 0.0
-8.284648895263672 10 32 0.9464688301086426 -2.268315553665161 0.0
4.418572425842285 22 7 1.0542056560516357 -5.088659286499023 0.0
3.820955753326416 4 -61 0.9609335660934448 8.974969863891602 0.0
-3.8306546211242676 47 67 1.0129806995391846 -0.6462407112121582 0.0
7.354991436004639 -46 74 0.9444237351417542 -5.094616413116455 0.0
-6.86061954498291 -43 36 0.9146308898925781 -5.423521041870117 0.0
1.609501838684082 -4 41 1.0275684595108032 2.6456308364868164 0.0
3.6342084407806396 -48 40 1.0946437120437622 2.7929651737213135 0.0
9.479679107666016 -10 66 0.9018594622612 -0.28626203536987305 0.0
9.121648788452148 -18 -93 0.9217327237129211 3.5531580448150635

In [88]:
# # Define parameters
# image_size = (512, 512)  # Size of the images
# output_dir = "Dataset/synthetic_eye_dataset_test_multiple"  # Output directory
# os.makedirs(output_dir, exist_ok=True)

# # plot_dir = "Dataset/synthetic_eye_dataset_test/plot"
# # os.makedirs(plot_dir, exist_ok=True)

# # generate synthetic images for each source test image
# generate_affine_transformed_images_multiple(image_list_test, 'dataset_eye_synth_test_multiple.csv', output_dir, num_images=2)